In [1]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [6]:
orig_data = pd.read_csv("jester-data-1.csv", header=None)

In [7]:
orig_data.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,...,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,...,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,...,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


In [8]:
data = orig_data.copy()

In [21]:
valid_count = (int)(data.shape[0] * 0.1)

In [23]:
data[-valid_count:] = 99

In [25]:
data.tail()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
24978,99,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
24979,99,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
24980,99,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
24981,99,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
24982,99,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0


In [41]:
train_set = data[:data.shape[0]-valid_count].values
valid_set = data[-valid_count:].values

In [42]:
print(train_set.shape)
print(valid_set.shape)

(22485, 101)
(2498, 101)


In [43]:
n_features = 10

latent_user_preferences = np.random.random((train_set.shape[0], n_features))
latent_jokes_features = np.random.random((train_set.shape[1], n_features))

In [44]:
print(latent_user_preferences.shape)
print(latent_jokes_features.shape)

(22485, 10)
(101, 10)


In [53]:
def predict_rating(user_id, joke_id):
    """ Predict a rating given a user_id and a joke_id.
    """
    user_preference = latent_user_preferences[user_id]
    joke_feature = latent_jokes_features[joke_id]
    
    return user_preference.dot(joke_feature)

def train(user_id, joke_id, rating, alpha = 0.0001):
    
    prediction_rating = predict_rating(user_id, joke_id)
    err =  (prediction_rating- rating);
    
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_jokes_features[joke_id]
    latent_jokes_features[joke_id] -= alpha * err * user_pref_values
    
    return err
    

def sgd(iterations = 300000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(0, iterations):
        error = []
        for user_id in range(0, latent_user_preferences.shape[0]):
            for joke_id in range(0, latent_jokes_features.shape[0]):
                rating = train_set[user_id][joke_id]
                if(rating != 99):
                    err = train(user_id, joke_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        print(mse)

In [54]:
sgd(100)

17.643944097566543
17.616112391545403
17.58557101564226
17.552052709117984
17.51526933253186
17.4749723926745
17.430971407043774


KeyboardInterrupt: 